In [ ]:
# Парсинг КБЖУ продуктов сайта Е-доставка

In [ ]:
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm

In [96]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [97]:
options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)

for i in range(1, 11): # Проверять количество страниц

    # https://edostavka.by/category/5138?page={i} Овощи и фрукты (2 страницы)
    # https://edostavka.by/category/5194?page={i} Молоко и яйца (10 страниц)
    # https://edostavka.by/category/5329?page={i} Хлеб и выпечка (3 страницы)
    # https://edostavka.by/category/5131?page={i} Мясо, птица, колбасы (5 страниц)
    # https://edostavka.by/category/5185?page={i} Рыба, морепродукты (3 страницы)
    # https://edostavka.by/category/5258?page={i} Замороженные продукты (3 страницы)
    # https://edostavka.by/category/5034?page={i} Крупы, макароны, сахар (8 страниц)
    # https://edostavka.by/category/5199?page={i} Масло, консервация, соусы (5 страниц)
    # https://edostavka.by/category/5091?page={i} Чипсы, орехи, снеки (3 страницы)
    # https://edostavka.by/category/5160?page={i} Сладости (7 страниц)
    # https://edostavka.by/category/4996?page={i} Здоровое питание (3 страницы)
    # https://edostavka.by/category/4974?page={i} Кулинария (2 страницы)
    
    driver.get(f'https://edostavka.by/category/5185?page={i}')
    
    html = driver.find_element(By.TAG_NAME, 'html')
    for n in range(100):
        html.send_keys(Keys.END)
    
    if driver.find_elements(By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'):
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'))).click()
        for n in range(100):
            html.send_keys(Keys.END)
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-image_link__lTrk0')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        with open(f'category_products_page/pages/page_{i}.html', 'w', encoding="utf-8") as file:
            file.write(str(soup))
    else:
        driver.execute_script("window.scrollTo(0, 1080)")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'card-image_link__lTrk0')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        with open(f'category_products_page/pages/page_{i}.html', 'w', encoding="utf-8") as file:
            file.write(str(soup))

driver.quit()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D7957D12+55474]
	(No symbol) [0x00007FF6D78C77C2]
	(No symbol) [0x00007FF6D777E0EB]
	(No symbol) [0x00007FF6D77BEBAC]
	(No symbol) [0x00007FF6D77BED2C]
	(No symbol) [0x00007FF6D77F9F77]
	(No symbol) [0x00007FF6D77DF19F]
	(No symbol) [0x00007FF6D77F7EF2]
	(No symbol) [0x00007FF6D77DEF33]
	(No symbol) [0x00007FF6D77B3D41]
	(No symbol) [0x00007FF6D77B4F84]
	GetHandleVerifier [0x00007FF6D7CBB762+3609346]
	GetHandleVerifier [0x00007FF6D7D11A80+3962400]
	GetHandleVerifier [0x00007FF6D7D09F0F+3930799]
	GetHandleVerifier [0x00007FF6D79F3CA6+694342]
	(No symbol) [0x00007FF6D78D2218]
	(No symbol) [0x00007FF6D78CE484]
	(No symbol) [0x00007FF6D78CE5B2]
	(No symbol) [0x00007FF6D78BEE13]
	BaseThreadInitThunk [0x00007FFD239B257D+29]
	RtlUserThreadStart [0x00007FFD2512AA78+40]


In [98]:
product_links = []
link = 'https://edostavka.by'
for file in os.listdir('category_products_page/pages'):
    soup = BeautifulSoup(open(f'category_products_page/pages/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    for card_div_tag in soup.find_all('div', class_='adult-wrapper_adult__eCCJW vertical_product__Q8mUI'):
        product_link = card_div_tag.find('a', class_='card-image_link__lTrk0').get('href')
        product_link_ready = link + product_link
        product_links.append(product_link_ready)

In [103]:
options = Options()
# options.add_argument('--headless')
# options.add_argument("--disable-infobars")
# options.add_argument("--disable-extensions")
options.add_argument("--start-maximized")
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(options=options)

for product_link in tqdm(product_links):
    product_id = product_link.split('https://edostavka.by/product/')[1]
    if not f'{product_id}.html' in os.listdir('category_products_page/product_pages'):        
        driver.get(product_link)
        if driver.find_elements(By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]') or driver.find_elements(By.XPATH, '/html/body/div[5]/div/div[2]/div/div/button[1]/span'):
            WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div/div/button[1]/span'))).click()
            # WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[2]/div/div[2]/button[2]'))).click()
            driver.execute_script("window.scrollTo(0, 700)")
            if driver.find_elements(By.XPATH, '//*[@id="tab_specifications"]'):
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab_specifications"]'))).click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            with open(f'category_products_page/product_pages/{product_id}.html', 'w', encoding="utf-8") as file:
                file.write(str(soup))
        else:
            driver.execute_script("window.scrollTo(0, 700)")
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab_specifications"]'))).click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            with open(f'category_products_page/product_pages/{product_id}.html', 'w', encoding="utf-8") as file:
                file.write(str(soup))

driver.quit()

  3%|▎         | 34/1090 [01:33<48:38,  2.76s/it] 


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6D7957D12+55474]
	(No symbol) [0x00007FF6D78C77C2]
	(No symbol) [0x00007FF6D777E0EB]
	(No symbol) [0x00007FF6D77BEBAC]
	(No symbol) [0x00007FF6D77BED2C]
	(No symbol) [0x00007FF6D77F9F77]
	(No symbol) [0x00007FF6D77DF19F]
	(No symbol) [0x00007FF6D77F7EF2]
	(No symbol) [0x00007FF6D77DEF33]
	(No symbol) [0x00007FF6D77B3D41]
	(No symbol) [0x00007FF6D77B4F84]
	GetHandleVerifier [0x00007FF6D7CBB762+3609346]
	GetHandleVerifier [0x00007FF6D7D11A80+3962400]
	GetHandleVerifier [0x00007FF6D7D09F0F+3930799]
	GetHandleVerifier [0x00007FF6D79F3CA6+694342]
	(No symbol) [0x00007FF6D78D2218]
	(No symbol) [0x00007FF6D78CE484]
	(No symbol) [0x00007FF6D78CE5B2]
	(No symbol) [0x00007FF6D78BEE13]
	BaseThreadInitThunk [0x00007FFD239B257D+29]
	RtlUserThreadStart [0x00007FFD2512AA78+40]


In [104]:
full_df = pd.DataFrame()

for file in tqdm(os.listdir('category_products_page/product_pages')):
    try:
        soup = BeautifulSoup(open(f'category_products_page/product_pages/{file}', 'r', encoding="utf-8").read(), 'html.parser')
    
        product_name = soup.find('h1', class_='typography typography_tag_h1 typography_weight_700 heading_heading__text__gP6AN heading_heading__text_level_1__7_duQ').text
        
        product_protein = re.sub(r'[,/:].+', ' ', re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[0])

        product_fats = re.sub(r'[,/:].+', ' ', re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[1])

        product_carbohydrate = re.sub(r'[,/:].+', ' ', re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[2])
        if product_carbohydrate in '/ ':
            product_carbohydrate = '0'

        product_calorie = re.sub(r'[,/:].+', ' ', re.sub(r'[А-я]', ' ', soup.find('div', class_='table_properties__SlSR7').text).split()[3])
        if product_calorie in '/ ':
            product_calorie = product_carbohydrate
            product_carbohydrate = '0'
               
        data = {'name':[product_name], 'protein':[product_protein], 'fats':[product_fats], 'carbohydrate':[product_carbohydrate], 'calorie':[product_calorie]}        

        df = pd.DataFrame(data=data)

        full_df = pd.concat([full_df, df])

    except:
        pass

100%|██████████| 299/299 [00:12<00:00, 23.65it/s]


In [105]:
full_df.to_csv('edostavka_products.csv', sep=';', index=False)